In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-w7p2/train.csv
/kaggle/input/2023-ml-w7p2/test.csv
/kaggle/input/2023-ml-w7p2/sample_submit.csv


In [2]:
train = pd.read_csv("/kaggle/input/2023-ml-w7p2/train.csv")
test = pd.read_csv("/kaggle/input/2023-ml-w7p2/test.csv")
submit = pd.read_csv("/kaggle/input/2023-ml-w7p2/sample_submit.csv")

In [3]:
train.head()

,index,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,1869,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
1,1,4528,9688-YGXVR,Female,0,No,No,44,Yes,No,...,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2,No
2,2,6344,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes
3,3,6739,6994-KERXL,Male,0,No,No,4,Yes,No,...,No,No,No,Yes,Month-to-month,Yes,Electronic check,55.90,238.5,No
4,4,432,2181-UAESM,Male,0,No,No,2,Yes,No,...,Yes,No,No,No,Month-to-month,No,Electronic check,53.45,119.5,No


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788 entries, 0 to 4787
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             4788 non-null   int64  
 1   Unnamed: 0        4788 non-null   int64  
 2   customerID        4788 non-null   object 
 3   gender            4788 non-null   object 
 4   SeniorCitizen     4788 non-null   int64  
 5   Partner           4788 non-null   object 
 6   Dependents        4788 non-null   object 
 7   tenure            4788 non-null   int64  
 8   PhoneService      4788 non-null   object 
 9   MultipleLines     4788 non-null   object 
 10  InternetService   4788 non-null   object 
 11  OnlineSecurity    4788 non-null   object 
 12  OnlineBackup      4788 non-null   object 
 13  DeviceProtection  4788 non-null   object 
 14  TechSupport       4788 non-null   object 
 15  StreamingTV       4788 non-null   object 
 16  StreamingMovies   4788 non-null   object 


In [5]:
train.drop(['index','Unnamed: 0','customerID'],axis=1,inplace=True)
test.drop(['index','Unnamed: 0','customerID'],axis=1,inplace=True)

In [6]:
# pd.to_numeric(train['TotalCharges'])

In [7]:
train.loc[train[train['TotalCharges'] == " "].index,['TotalCharges']] = np.nan
test.loc[test[test['TotalCharges'] == " "].index,['TotalCharges']] = np.nan

In [8]:
train['TotalCharges']=train['TotalCharges'].astype('float64')
test['TotalCharges']=test['TotalCharges'].astype('float64')

In [9]:
x=train.drop(['Churn'],axis=1)
y=train[['Churn']]

In [10]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4788 entries, 0 to 4787
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            4788 non-null   object 
 1   SeniorCitizen     4788 non-null   int64  
 2   Partner           4788 non-null   object 
 3   Dependents        4788 non-null   object 
 4   tenure            4788 non-null   int64  
 5   PhoneService      4788 non-null   object 
 6   MultipleLines     4788 non-null   object 
 7   InternetService   4788 non-null   object 
 8   OnlineSecurity    4788 non-null   object 
 9   OnlineBackup      4788 non-null   object 
 10  DeviceProtection  4788 non-null   object 
 11  TechSupport       4788 non-null   object 
 12  StreamingTV       4788 non-null   object 
 13  StreamingMovies   4788 non-null   object 
 14  Contract          4788 non-null   object 
 15  PaperlessBilling  4788 non-null   object 
 16  PaymentMethod     4788 non-null   object 


In [11]:
data=pd.concat([x,test],axis=0)

In [12]:
data=data.to_dict('records')
x=x.to_dict('records')
test=test.to_dict('records')

In [13]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer()
dv.fit(data)
x=dv.transform(x)
test=dv.transform(test)
data=dv.transform(data)

In [14]:
from sklearn.impute import SimpleImputer
si = SimpleImputer()
si.fit(data)
x=si.transform(x)
test=si.transform(test)

In [15]:
y['Churn'].unique()

array(['No', 'Yes'], dtype=object)

In [16]:
y['Churn'].replace({'No':0,'Yes':1},inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [17]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(x,y['Churn'])

LogisticRegression(max_iter=1000)

In [18]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y['Churn'],model.predict(x)))

0.8057644110275689


In [19]:
submit['Churn'] = model.predict(test)
submit.to_csv("submit.csv",mode='w',header=True,index=False)